In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv("Dataset.csv")

# Drop useless columns
df = df.drop(columns=["Unnamed: 0", "EtCO2"], errors="ignore")

# Remove rows with missing structure
df = df.dropna(subset=["Patient_ID", "SepsisLabel"])

# Convert types
df["Patient_ID"] = df["Patient_ID"].astype(int)
df["SepsisLabel"] = df["SepsisLabel"].astype(int)

# Sort time-series correctly
df = df.sort_values(["Patient_ID", "Hour"])

print("Shape:", df.shape)

Shape: (546122, 42)


In [5]:
def create_early_label(group, shift=5):
    group = group.copy()
    group["EarlyLabel"] = 0
    
    if group["SepsisLabel"].sum() > 0:
        t = group[group["SepsisLabel"] == 1]["Hour"].min()
        group.loc[
            (group["Hour"] >= t - shift) &
            (group["Hour"] < t),
            "EarlyLabel"
        ] = 1
        
    return group

df = df.groupby("Patient_ID", group_keys=False).apply(create_early_label)

C:\Users\SELVAGANAPATHI\AppData\Local\Temp\ipykernel_3256\830231438.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Patient_ID", group_keys=False).apply(create_early_label)


In [6]:
patients = df["Patient_ID"].unique()

train_ids, test_ids = train_test_split(
    patients,
    test_size=0.2,
    random_state=42
)

train_df = df[df["Patient_ID"].isin(train_ids)].copy()
test_df = df[df["Patient_ID"].isin(test_ids)].copy()

print("Train patients:", len(train_ids))
print("Test patients:", len(test_ids))

Train patients: 11245
Test patients: 2812


In [7]:
train_df = (
    train_df
    .groupby("Patient_ID", as_index=False)
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
)

test_df = (
    test_df
    .groupby("Patient_ID", as_index=False)
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
)

medians = train_df.median(numeric_only=True)

train_df = train_df.fillna(medians)
test_df = test_df.fillna(medians)

C:\Users\SELVAGANAPATHI\AppData\Local\Temp\ipykernel_3256\1481954895.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())
C:\Users\SELVAGANAPATHI\AppData\Local\Temp\ipykernel_3256\1481954895.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())


In [8]:
df.head()

,Hour,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_ID,EarlyLabel
132940,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,83.14,0.0,NaN,NaN,-0.03,1.0,0,1,0
132941,1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,...,NaN,83.14,0.0,NaN,NaN,-0.03,2.0,0,1,0
132942,2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,...,NaN,83.14,0.0,NaN,NaN,-0.03,3.0,0,1,0
132943,3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,24.0,NaN,...,NaN,83.14,0.0,NaN,NaN,-0.03,4.0,0,1,0
132944,4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,...,NaN,83.14,0.0,NaN,NaN,-0.03,5.0,0,1,0


In [9]:
def add_features(df):
    
    df = df.sort_values(["Patient_ID", "Hour"])
    
    # Shock Index
    df["ShockIndex"] = df["HR"] / df["SBP"]
    
    # Deltas
    df["HR_delta"] = df.groupby("Patient_ID")["HR"].diff()
    df["MAP_delta"] = df.groupby("Patient_ID")["MAP"].diff()
    df["Lactate_delta"] = df.groupby("Patient_ID")["Lactate"].diff()
    
    # Rolling Means
    df["HR_roll3"] = (
        df.groupby("Patient_ID")["HR"]
          .rolling(3)
          .mean()
          .reset_index(level=0, drop=True)
    )
    
    df["MAP_roll3"] = (
        df.groupby("Patient_ID")["MAP"]
          .rolling(3)
          .mean()
          .reset_index(level=0, drop=True)
    )
    
    return df

# train_df = add_features(train_df)
# test_df = add_features(test_df)

In [10]:
def add_temporal_features(df):

    df = df.sort_values(["Patient_ID", "Hour"])

    for col in ["HR", "MAP", "Lactate"]:
        
        df[f"{col}_roll6"] = (
            df.groupby("Patient_ID")[col]
              .rolling(6)
              .mean()
              .reset_index(level=0, drop=True)
        )

        df[f"{col}_std6"] = (
            df.groupby("Patient_ID")[col]
              .rolling(6)
              .std()
              .reset_index(level=0, drop=True)
        )

    return df

train_df = add_temporal_features(train_df)
test_df = add_temporal_features(test_df)

In [11]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [12]:
EXCLUDE = ["Patient_ID", "Hour", "SepsisLabel", "EarlyLabel"]

features = [col for col in train_df.columns if col not in EXCLUDE]

print("Total features:", len(features))

Total features: 45


In [13]:
pos = train_df["EarlyLabel"].sum()
neg = len(train_df) - pos

scale_weight = neg / pos

print("Positive:", pos)
print("Negative:", neg)

Positive: 4234
Negative: 435148


In [14]:
model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    scale_pos_weight=scale_weight,
    eval_metric="aucpr",
    random_state=42
)

model.fit(train_df[features], train_df["EarlyLabel"])

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'aucpr'


In [ ]:
calibrated_model.predict_proba()

In [15]:
probs = model.predict_proba(test_df[features])[:, 1]

auroc = roc_auc_score(test_df["EarlyLabel"], probs)
auprc = average_precision_score(test_df["EarlyLabel"], probs)

print("AUROC:", auroc)
print("AUPRC:", auprc)

AUROC: 0.7739725663338535
AUPRC: 0.038443575601797175


In [16]:
baseline = test_df["EarlyLabel"].mean()
print("Baseline prevalence:", baseline)

Baseline prevalence: 0.009115608019486602


In [35]:
# Check available patient IDs in test set
available_patients = test_df["Patient_ID"].unique()
print(f"Available patients in test set: {len(available_patients)}")
print(f"First few patients: {available_patients[:10]}")

# Use first patient with data
patient_id = available_patients[0]
print(f"\nUsing Patient ID: {patient_id}")

Available patients in test set: 2812
First few patients: [15 23 24 30 49 55 59 60 67 69]

Using Patient ID: 15


In [63]:
patient_id = 20563  # example

patient_data = test_df[test_df["Patient_ID"] == patient_id]
patient_data = patient_data.sort_values("Hour")

In [ ]:
import time

def simulate_patient_stream(patient_df, model, features, threshold):
    
    print(f"Starting ICU Twin Simulation for Patient {patient_df['Patient_ID'].iloc[0]}")
    
    risk_history = []
    
    for i in range(len(patient_df)):
        
        current_row = patient_df.iloc[:i+1]
        latest_hour = current_row.iloc[-1]
        
        # Generate features for latest hour
        X_current = latest_hour[features].values.reshape(1, -1)
        
        # Predict risk probability (using predict_proba instead of predict)
        risk = model.predict_proba(X_current)[0, 1]
        risk_history.append(risk)
        
        print(f"Hour {int(latest_hour['Hour'])} | Risk: {risk:.3f}")
        
        if risk > threshold:
            print("⚠ ALERT: High Sepsis Risk Detected")
        
        time.sleep(0.3)  # simulate delay
    
    return risk_history

In [96]:
import time
import numpy as np

def simulate_patient_stream(patient_df, model, features,
                            yellow_threshold=0.35,
                            red_threshold=0.50,
                            min_persistence=2):
    
    print(f"\nStarting ICU Twin Simulation for Patient {patient_df['Patient_ID'].iloc[0]}")
    
    risk_history = []
    alert_counter = 0
    
    for i in range(len(patient_df)):
        
        latest_hour = patient_df.iloc[i]
        X_current = latest_hour[features].values.reshape(1, -1)
        
        risk = model.predict_proba(X_current)[0, 1]
        risk_history.append(risk)
        
        # Compute short-term trend (last 3 hours)
        if len(risk_history) >= 3:
            recent_avg = np.mean(risk_history[-3:])
            escalation = risk - recent_avg
        else:
            escalation = 0
        
        # Determine risk level
        if risk >= red_threshold:
            level = "🔴 HIGH"
        elif risk >= yellow_threshold:
            level = "🟡 MODERATE"
        else:
            level = "🟢 STABLE"
        
        print(f"Hour {int(latest_hour['Hour'])} | Risk: {risk:.3f} | Level: {level}")
        
        # Persistent alert logic
        if risk >= red_threshold:
            alert_counter += 1
        else:
            alert_counter = 0
        
        # Trigger alert only if persistent
        if alert_counter >= min_persistence:
            print("⚠ ALERT: Sustained High Sepsis Risk Detected")
        
        # Optional: Trend-based alert
        if escalation > 0.08 and risk > yellow_threshold:
            print("📈 Escalating Risk Detected")
        
        time.sleep(0.2)
    
    return risk_history

In [97]:
risk_trend = simulate_patient_stream(patient_data, model, features)


Starting ICU Twin Simulation for Patient 20563
Hour 0 | Risk: 0.626 | Level: 🔴 HIGH
Hour 1 | Risk: 0.579 | Level: 🔴 HIGH
⚠ ALERT: Sustained High Sepsis Risk Detected
Hour 2 | Risk: 0.579 | Level: 🔴 HIGH
⚠ ALERT: Sustained High Sepsis Risk Detected
Hour 3 | Risk: 0.546 | Level: 🔴 HIGH
⚠ ALERT: Sustained High Sepsis Risk Detected
Hour 4 | Risk: 0.540 | Level: 🔴 HIGH
⚠ ALERT: Sustained High Sepsis Risk Detected
Hour 5 | Risk: 0.499 | Level: 🟡 MODERATE
Hour 6 | Risk: 0.483 | Level: 🟡 MODERATE
Hour 7 | Risk: 0.483 | Level: 🟡 MODERATE
Hour 8 | Risk: 0.481 | Level: 🟡 MODERATE
Hour 9 | Risk: 0.477 | Level: 🟡 MODERATE
Hour 10 | Risk: 0.479 | Level: 🟡 MODERATE
Hour 11 | Risk: 0.512 | Level: 🔴 HIGH
Hour 12 | Risk: 0.448 | Level: 🟡 MODERATE
Hour 13 | Risk: 0.335 | Level: 🟢 STABLE
Hour 14 | Risk: 0.400 | Level: 🟡 MODERATE
Hour 15 | Risk: 0.381 | Level: 🟡 MODERATE
Hour 16 | Risk: 0.499 | Level: 🟡 MODERATE
Hour 17 | Risk: 0.399 | Level: 🟡 MODERATE
Hour 18 | Risk: 0.402 | Level: 🟡 MODERATE
Hour 19 | 

In [ ]:
print("All Patient IDs in test set:")
print(sorted(test_df["Patient_ID"].unique()))
print(f"\nTotal: {len(test_df['Patient_ID'].unique())} unique patients")

All Patient IDs in test set:
[15, 23, 24, 30, 49, 55, 59, 60, 67, 69, 76, 80, 93, 97, 100, 105, 107, 111, 122, 124, 137, 141, 145, 147, 148, 149, 155, 165, 166, 174, 185, 195, 214, 223, 226, 256, 259, 260, 267, 273, 280, 281, 287, 288, 313, 332, 336, 340, 343, 344, 347, 365, 382, 384, 385, 389, 412, 431, 441, 445, 449, 453, 454, 458, 467, 474, 475, 483, 493, 496, 498, 503, 507, 511, 520, 531, 543, 548, 554, 564, 573, 580, 586, 590, 598, 600, 601, 612, 614, 615, 626, 630, 634, 636, 638, 641, 655, 659, 667, 669, 673, 677, 681, 689, 691, 695, 700, 707, 709, 713, 729, 744, 745, 760, 773, 779, 781, 784, 788, 791, 803, 804, 808, 815, 826, 839, 848, 855, 858, 872, 877, 883, 885, 903, 916, 925, 938, 942, 948, 954, 971, 977, 980, 992, 1007, 1012, 1017, 1023, 1029, 1036, 1039, 1044, 1053, 1057, 1062, 1064, 1068, 1069, 1072, 1075, 1077, 1078, 1092, 1094, 1095, 1104, 1135, 1136, 1159, 1160, 1167, 1169, 1171, 1183, 1189, 1192, 1196, 1210, 1221, 1231, 1242, 1246, 1251, 1265, 1280, 1283, 1286, 1290, 

In [48]:
# Get patient IDs with SepsisLabel = 1
sepsis_patients = test_df[test_df["SepsisLabel"] == 1]["Patient_ID"].unique()
print("Patient IDs with SepsisLabel = 1:")
print(sorted(sepsis_patients))
print(f"\nTotal: {len(sepsis_patients)} patients with sepsis")

Patient IDs with SepsisLabel = 1:
[15, 141, 185, 226, 260, 384, 483, 601, 614, 784, 872, 938, 1069, 1072, 1092, 1183, 1192, 1280, 1291, 1307, 1319, 1370, 1384, 1389, 1433, 1446, 1666, 1839, 1849, 2029, 2161, 2213, 2273, 2292, 2321, 2332, 2399, 2404, 2441, 2720, 2852, 2853, 2905, 2974, 3018, 3073, 3158, 3182, 3325, 3373, 3390, 3486, 3497, 3532, 3668, 3806, 3809, 4243, 4326, 4329, 4376, 4440, 4630, 4698, 4740, 4824, 4868, 5026, 5188, 5191, 5211, 5251, 5277, 5519, 5554, 5566, 5610, 5637, 5691, 5765, 5772, 5841, 5906, 5932, 5943, 5955, 5981, 6051, 6094, 6474, 6573, 6669, 6770, 6787, 6995, 7093, 7128, 7283, 7552, 8205, 8255, 8275, 8316, 8372, 8460, 8470, 8603, 8608, 8781, 8802, 8814, 9146, 9278, 9313, 9501, 9541, 9569, 9671, 9862, 9894, 10280, 10490, 10631, 10700, 10771, 10880, 10954, 10957, 10974, 10997, 10998, 11251, 11290, 11319, 11470, 11551, 11565, 11720, 11876, 12043, 12091, 12117, 12138, 12194, 12293, 12303, 12333, 12461, 12525, 12682, 12732, 12733, 13201, 13346, 13542, 13569, 13640,

In [71]:
from sklearn.metrics import roc_curve
import numpy as np

# Use actual data from the model
y_true = test_df["EarlyLabel"]
y_probs = probs

fpr, tpr, thresholds = roc_curve(y_true, y_probs)

In [72]:
# Calculate Youden's J statistic to find optimal threshold
j_scores = tpr - fpr
best_idx = np.argmax(j_scores)
best_threshold = thresholds[best_idx]

print("Best threshold (Youden's J):", best_threshold)
print(f"Sensitivity (TPR) at best threshold: {tpr[best_idx]:.4f}")
print(f"Specificity (1-FPR) at best threshold: {1-fpr[best_idx]:.4f}")

Best threshold (Youden's J): 0.32337236
Sensitivity (TPR) at best threshold: 0.6413
Specificity (1-FPR) at best threshold: 0.7710


In [ ]:
# Find threshold for target sensitivity (real sepsis true positive rate)
target_sensitivity = 0.75 
idx = np.where(tpr >= target_sensitivity)[0][0]

threshold_for_sensitivity = thresholds[idx]

print(f"Threshold for {target_sensitivity*100:.0f}% sensitivity: {threshold_for_sensitivity:.4f}")
print(f"Specificity at this threshold: {1-fpr[idx]:.4f}")

Threshold for 75% sensitivity: 0.2445
Specificity at this threshold: 0.6568


In [90]:
target_sensitivity = 0.57

valid_idxs = np.where(tpr >= target_sensitivity)[0]

if len(valid_idxs) > 0:
    idx = valid_idxs[-1]   # use last index
    threshold_for_sensitivity = thresholds[idx]

    print("Threshold:", threshold_for_sensitivity)
    print("Sensitivity:", tpr[idx])
    print("Specificity:", 1 - fpr[idx])

Threshold: 0.000590047
Sensitivity: 1.0
Specificity: 0.0
